In [1]:

import os

In [2]:
%pwd

'e:\\DeepLearning\\Kidney-Disease-Classification\\research'

In [3]:

os.chdir("../")

In [14]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:

from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
project_dir = Path(r"E:\DeepLearning\Kidney-Disease-Classification")
ROOT_DIR = Path.cwd()
CONFIG_FILE_PATH = ROOT_DIR / "config" / "config.yaml"
PARAMS_FILE_PATH = ROOT_DIR / "params.yaml"

In [8]:
ROOT_DIR


WindowsPath('e:/DeepLearning/Kidney-Disease-Classification')

In [9]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:

import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [11]:

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [15]:

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-30 13:46:00,809: INFO: common: yaml file: e:\DeepLearning\Kidney-Disease-Classification\config\config.yaml loaded successfully]
[2025-07-30 13:46:00,812: INFO: common: yaml file: e:\DeepLearning\Kidney-Disease-Classification\params.yaml loaded successfully]
[2025-07-30 13:46:00,813: INFO: common: created directory at: artifacts]
[2025-07-30 13:46:00,815: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-30 13:46:00,816: INFO: 1060046524: Downloading data from https://drive.google.com/file/d/1MLceRM-ckqxHoJLQYgAkJ_8WKdE2H4o-/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1MLceRM-ckqxHoJLQYgAkJ_8WKdE2H4o-
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1MLceRM-ckqxHoJLQYgAkJ_8WKdE2H4o-&confirm=t&uuid=e68802ea-34f5-484e-9eb1-95edd9902021
To: e:\DeepLearning\Kidney-Disease-Classification\artifacts\data_ingestion\data.zip
100%|██████████| 501M/501M [01:10<00:00, 7.07MB/s] 

[2025-07-30 13:47:14,961: INFO: 1060046524: Downloaded data from https://drive.google.com/file/d/1MLceRM-ckqxHoJLQYgAkJ_8WKdE2H4o-/view?usp=sharing into file artifacts/data_ingestion/data.zip]
